In [27]:
import json
import pandas as pd
from tensorflow.keras.models import load_model
import numpy as np
import joblib
from sklearn.preprocessing import MinMaxScaler

In [4]:
# Load JSON file
with open('encoding_maps.json', 'r') as file:
    encoding_maps = json.load(file)

# Load JSON file
with open('drop_columns.json', 'r') as file:
    drop_columns = json.load(file)

In [5]:
test_data = pd.read_csv('kddcup.testdata.unlabeled.gz', header=None)

In [6]:
# Load the feature names and attack types
with open('kddcup.names.txt', 'r') as f:
    feature_names = [line.strip() for line in f.readlines() if not line.startswith('|') and line.strip()][1:]

In [7]:
feature_names = [name.split(':')[0] for name in feature_names]

In [8]:
test_data.columns = feature_names

In [9]:
test_data['protocol_type'] = test_data['protocol_type'].map(encoding_maps['protocol_type_map'])
test_data['service'] = test_data['service'].map(encoding_maps['service_map'])
test_data['flag'] = test_data['flag'].map(encoding_maps['flag_map'])

In [10]:
test_data = test_data.drop(drop_columns, axis=1)

In [ ]:
test_data.isna().sum()

In [19]:
test_data.dropna(inplace=True)

In [11]:
# Load Models
models = {
    'NN': load_model('models/NN_model.h5'),
    'LGBM': joblib.load('models/LGBM_model.pkl'),
    'CatB': joblib.load('models/CatB_model.pkl'),
    'LR': joblib.load('models/LR_model.pkl'),
    'RF': joblib.load('models/RF_model.pkl'),
    'XGB': joblib.load('models/XGB_model.pkl')
}

In [12]:
# NN Predictions
predictions = np.argmax(models['NN'].predict(test_data), axis=1)

93255/93255 ━━━━━━━━━━━━━━━━━━━━ 67s 717us/step


In [28]:
sc = MinMaxScaler()
X = sc.fit_transform(test_data)

In [30]:
predictions = models['LGBM'].predict(X)

In [31]:
predictions = models['CatB'].predict(X)

In [32]:
predictions = models['LR'].predict(X)

In [33]:
predictions = models['RF'].predict(X)

In [34]:
predictions = models['XGB'].predict(X)